In [1]:
import nltk
import pandas as pd
import numpy as np
import re
import codecs
import json

import re
import numpy as np
import pandas as pd
from pprint import pprint
import pickle
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.word2vec import LineSentence
from gensim.models.ldamulticore import LdaMulticore
# spacy for lemmatization
import spacy
# NLTK for text cleaning
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords, names
from nltk.tokenize import RegexpTokenizer
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
# TextBlob package for translation and spelling correction
from textblob import TextBlob

nlp = spacy.load('en')
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/home/argha/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/argha/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
nydata = pd.read_json('/home/argha/Dropbox/data-files/reviews/NY_State_hotels_reviews.json',
                    lines=True, encoding='utf-8')
calidata = pd.read_json('/home/argha/Dropbox/data-files/reviews/CaliforniaState_hotels_reviews.json',
                    lines=True, encoding='utf-8')

In [3]:
nydata.head(1)

,Cleanliness,Comfort,Facilities,FreeWifi,GivenScore,Heading,HotelName,Location,NegativeReview,NumReviewGiven,PositiveReview,SelfTag,Staff,StayDate,UserAgeGroup,UserCountry,UserName,ValueForMoney
0,[9.9],[10],[9.8],[8],[\n10\n],[Would love to stay age. Let me know of any s...,[Adelphi Hotel],[9.9],[Not a thing.],[\n1 review\n],[I really enjoyed the personal. I live locall...,"[\n, Leisure trip\n, \n, Couple\n, \n, King...",[9.6],[\nStayed in January 2019\n],[\n],[USA],[Rose],[9.3]


In [4]:
def get_text(rev):
    return rev[0] if str(rev)!='nan' else ''

In [7]:
ny_df = nydata[['HotelName','PositiveReview','NegativeReview','StayDate']].copy().applymap(get_text)
cal_df = calidata[['HotelName','PositiveReview','NegativeReview','StayDate']].copy().applymap(get_text)
demo_data = pd.concat([ny_df,cal_df],ignore_index=True)
demo_data['StayDate'] = demo_data['StayDate'].apply(lambda x: x.replace('\n','')).apply(lambda x: x.replace('Stayed in ',''))

demo_data.to_json('demo_data.json', orient='columns')
demo_data = pd.read_json('demo_data.json',orient='columns', encoding='utf-8')
demo_data.sample()

,HotelName,PositiveReview,NegativeReview,StayDate
1052246,Studio Inn & Suites Downey,I liked the room it was clean carpet was a plu...,,


In [8]:
demo_data.sample()

,HotelName,PositiveReview,NegativeReview,StayDate
271270,Hotel Stanford NYC,,Friendliness of staff\nOverall condition - pre...,July 2018


In [ ]:
ny_df.sample()

In [ ]:
ny_df = nydata[['HotelName','PositiveReview','NegativeReview','StayDate']].copy().applymap(get_text)
cal_df = calidata[['HotelName','PositiveReview','NegativeReview','StayDate']].copy().applymap(get_text)

ny_df['FullReview'] = [pos+' '+neg for pos,neg in zip(ny_df['PositiveReview'],ny_df['NegativeReview'])]
cal_df['FullReview'] = [pos+' '+neg for pos,neg in zip(cal_df['PositiveReview'],cal_df['NegativeReview'])]

In [ ]:
all_data = pd.concat([ny_df,cal_df],ignore_index=True)
all_data['StayDate'] = all_data['StayDate'].apply(lambda x: x.replace('\n','')).apply(lambda x: x.replace('Stayed in ',''))

In [3]:
sampledf = nydata.head(n=1000)
sampledf.shape

(1000, 18)

In [13]:
sampledf.sample()

,Cleanliness,Comfort,Facilities,FreeWifi,GivenScore,Heading,HotelName,Location,NegativeReview,NumReviewGiven,PositiveReview,SelfTag,Staff,StayDate,UserAgeGroup,UserCountry,UserName,ValueForMoney
971,[9.8],[9.7],[9.4],[8.8],[\n10\n],[Would definitely return.],"[Pine Knoll Lodge & Cabins, Inc.]",[9.8],NaN,[\n1 review\n],[Homey. Very friendly staff. Updated rooms],"[\n, Leisure trip\n, \n, Couple\n, \n, Room...",[9.7],[\nStayed in August 2017\n],[\n],[USA],[Robert],[8.9]


In [4]:
def get_text(rev):
    return rev[0] if str(rev)!='nan' else ''

def review_cleaned(review_df):
    df = review_df[['HotelName','PositiveReview','NegativeReview','StayDate']].copy().applymap(get_text)
    df['FullReview'] = [pos+' '+neg for pos,neg in zip(df['PositiveReview'],df['NegativeReview'])]
    df['StayDate'] = df['StayDate'].apply(lambda x: x.replace('\n','')).apply(lambda x: x.replace('Stayed in ',''))
    return df

def review_to_sentence(df):
    all_sentences = []
    from nltk.tokenize import sent_tokenize
    import pandas as pd
    allreview = df['FullReview']
    for areview in allreview:
        all_sentences.extend(sent_tokenize(areview))
    tokensentence = pd.DataFrame(data=all_sentences,columns=['TokenSentence'])
    return tokensentence

def sentence_sentiment(text):
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    analyzer = SentimentIntensityAnalyzer()
    compound_sentiment = analyzer.polarity_scores(text)['compound']
    return compound_sentiment

def token_to_sentiment(df):
    df['CompoundSentiment'] = df['TokenSentence'].apply(sentence_sentiment)
    return df

In [5]:
text = review_cleaned(sampledf)
tokensentence = review_to_sentence(text)
sentencesentiment = token_to_sentiment(tokensentence)
# sentencesentiment.sample()

In [6]:
mallet_lda_topics={
                0:'Hotel Staff',
                1:'Accessibility',
                2:'Food',
                3:'Overall Experience',
                4:'Noise',
                5:'Value for Money',
                6:'Room Amenities',
                7:'Location in the city',
                8:'Overall Experience',
                9:'Cleanliness',
                10:'Early Check-in/Late Check-out',
                11:'Health and Wellness Amenities',
                12:'Booking Experience',
                13:'Sleep Quality',
                14:'Parking Facility'
            }

In [7]:
# helper functions for text preprocessing & LDA modeling:

def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """

    return token.is_punct or token.is_space or token.like_num or token.is_digit

def line_review(filename):
    """
    generator function to read in reviews from Pandas Series
    and un-escape the original line breaks in the text
    """

    #with codecs.open(filename, encoding='utf_8') as f:
    for review in filename:
        yield review.replace('\\n', '\n')

def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """

    for parsed_review in nlp.pipe(line_review(filename), batch_size=10000, n_threads=10):
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    # load finished dictionary from disk
    trigram_dictionary = Dictionary.load('./models2/trigram_dict_all.dict')

    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)
        
def nCPU():
    import multiprocessing
    N = multiprocessing.cpu_count()-1
    return N

In [8]:
def topic_extractor(df, min_topic_freq=0.10):
    from tqdm import tqdm
    from operator import itemgetter
    ncpu = nCPU()
    dfc=df.copy()
    text = dfc['TokenSentence'].copy()
    trigram_dictionary = Dictionary.load('./models2/trigram_dict_all.dict')
    lda = LdaMulticore.load('./models2/mallet_lda_model')
    trigram_review = LineSentence('./tri_temporary.txt')
    bigram_model = Phrases.load('./models2/bigram_model.txt')
    trigram_model = Phrases.load('./models2/trigram_model.txt')
    topic_list = []
    trigram_list = []
    freq_list = []
    # parse the review text with spaCy
    for parsed_review in tqdm(nlp.pipe(line_review(text),
                                    batch_size=10000, n_threads=ncpu)):
        # lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review
                            if not punct_space(token)]
        # apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]

        common_terms = ['-PRON-','hotel'] #'service',
        # remove any remaining stopwords
        trigram_review = [term for term in trigram_review
                            if term not in spacy.lang.en.stop_words.STOP_WORDS]
        trigram_review = [term for term in trigram_review
                            if term not in common_terms]
        if len(trigram_review)==0:
            topic_number=-1
            freq = 0.0
            tri = str([])
        else:
            # create a bag-of-words representation
            review_bow = trigram_dictionary.doc2bow(trigram_review)
            # create an LDA representation
            review_lda = lda.get_document_topics(review_bow)
            # print the most highly related topic name and frequency
            review_lda = sorted(review_lda, key=itemgetter(1),reverse=True)[0]
            topic_number = review_lda[0]
            freq = review_lda[1]
            if freq < min_topic_freq:
                topic_number=-1
                freq = 0.0
            
        topic_list.append(topic_number)
        freq_list.append(round(freq,2))
        trigram_list.append(trigram_review)
    dfc['Topic']=topic_list
    dfc['TopicFrequency']=freq_list
    dfc['Trigram']=trigram_list
    return dfc


In [9]:
topicdf = topic_extractor(sentencesentiment)

4058it [00:10, 391.32it/s]


topicdf.head()

In [10]:
def topic_scorer(df):
    xdf = pd.get_dummies(df,prefix='Topic', 
                     prefix_sep='_', dummy_na=False, 
                     columns=['Topic'])
    topics = ['Topic_0', 'Topic_1', 'Topic_2','Topic_3',
              'Topic_4', 'Topic_5', 'Topic_6', 'Topic_7', 
              'Topic_8','Topic_9', 'Topic_10', 'Topic_11',
              'Topic_12', 'Topic_13','Topic_14']
    topic_dict = {}
    for atopic in topics:
        if atopic in xdf.columns.values:
            xdf[atopic] = xdf[atopic] * xdf.CompoundSentiment
            m = np.mean(list(filter(lambda a: a != 0, xdf[atopic])))
    #         print(mallet_lda_topics[int(atopic.replace('Topic_',''))],round(m,2))
            topic_dict[mallet_lda_topics[int(atopic.replace('Topic_',''))]] = round(m,2)
        else:
            topic_dict[mallet_lda_topics[int(atopic.replace('Topic_',''))]] = 'No information available'
    return topic_dict

In [11]:
dd = topic_scorer(topicdf)

In [12]:
for key in dd:
    print(key,dd[key])

Hotel Staff 0.71
Accessibility 0.67
Food 0.52
Overall Experience 0.52
Noise 0.03
Value for Money 0.46
Room Amenities 0.15
Location in the city 0.54
Cleanliness 0.1
Early Check-in/Late Check-out 0.22
Health and Wellness Amenities 0.66
Booking Experience 0.12
Sleep Quality 0.41
Parking Facility 0.32


In [ ]:
odf=topicdf[['CompoundSentiment','Topic']].copy()

In [ ]:
xdf = pd.get_dummies(odf,prefix='Topic', 
                     prefix_sep='_', dummy_na=False, 
                     columns=['Topic'])

In [ ]:
xdf.head()

In [ ]:
topics = ['Topic_0', 'Topic_1', 'Topic_2',
       'Topic_3', 'Topic_4', 'Topic_5', 'Topic_6', 'Topic_7', 'Topic_8',
       'Topic_9', 'Topic_10', 'Topic_11', 'Topic_12', 'Topic_13',
       'Topic_14']
for atopic in topics:
    xdf[atopic] = xdf[atopic] * xdf.CompoundSentiment

In [ ]:
xdf.head()

In [ ]:
for atopic in topics:
    m = np.mean(list(filter(lambda a: a != 0, xdf[atopic])))
#     print(mallet_lda_topics[int(atopic.replace('Topic_',''))],round(m,2))
    topic_dict[mallet_lda_topics[int(atopic.replace('Topic_',''))]] = round(m,2)

In [7]:
nystar_data = pd.read_json('/home/argha/Dropbox/data-files/hotel_info/NewYorkState_hotels_stars.json', lines=True,encoding='utf-8')
calistar_data = pd.read_json('/home/argha/Dropbox/data-files/hotel_info/CaliforniaState_hotels_stars.json',lines=True, encoding='utf-8')